# TCC mineração de dados nos dados da PRF

#### Analise e insights 

In [1]:

import pandas as pd 
import folium as fol
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pyspark

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentes_filtrados_ES.csv'
dataSet = pd.read_csv(file_path, encoding='ISO-8859-1')


Mapa de calor dos acidentes nas rodovias federais do ES

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentes_filtrados_ES.csv'

try:
    
    dataSet = pd.read_csv(file_path, delimiter=',', encoding='ISO-8859-1')

    dataSet['latitude'] = dataSet['latitude'].astype(str).str.replace(',', '.')
    dataSet['longitude'] = dataSet['longitude'].astype(str).str.replace(',', '.')

    dataSet['latitude'] = pd.to_numeric(dataSet['latitude'], errors='coerce')
    dataSet['longitude'] = pd.to_numeric(dataSet['longitude'], errors='coerce')

    data_filtered = dataSet.dropna(subset=['latitude', 'longitude'])

    heat_data = list(zip(data_filtered['latitude'], data_filtered['longitude']))

    map_center = [data_filtered['latitude'].mean(), data_filtered['longitude'].mean()]
    m = folium.Map(location=map_center, zoom_start=8)

    HeatMap(heat_data).add_to(m)

    output_path = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/mapa_de_calor_acidentes.html"
    m.save(output_path)

    print(f"Mapa de calor gerado, salvo como {output_path}")

except Exception as e:
    print(f"Erro ao gerar o mapa de calor: {e}")


Mapa de calor gerado com sucesso! Salvo como C:/Users/gusta/Desktop/TCC Gustavo/notebooks/mapa_de_calor_acidentes.html


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

try:
    
    file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentes_filtrados_ES.csv'
    
    dataSet = pd.read_csv(file_path, encoding='ISO-8859-1')

    dataSet['latitude'] = dataSet['latitude'].astype(str).str.replace(',', '.')
    dataSet['longitude'] = dataSet['longitude'].astype(str).str.replace(',', '.')

    dataSet['latitude'] = pd.to_numeric(dataSet['latitude'], errors='coerce')
    dataSet['longitude'] = pd.to_numeric(dataSet['longitude'], errors='coerce')

    data_filtered = dataSet.dropna(subset=['latitude', 'longitude'])

    map_center = [data_filtered['latitude'].mean(), data_filtered['longitude'].mean()]
    m = folium.Map(location=map_center, zoom_start=8)

    marker_cluster = MarkerCluster().add_to(m)

    # Adicionar pontos ao cluster
    for _, row in data_filtered.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=(
                f"<b>Município:</b> {row['municipio']}<br>"
                f"<b>BR:</b> {row['br']}<br>"
                f"<b>Data:</b> {row['data_inversa']}<br>"
                f"<b>Latitude:</b> {row['latitude']}<br>"
                f"<b>Longitude:</b> {row['longitude']}"
            ),
        ).add_to(marker_cluster)

    output_path = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/mapa_de_pontos_cluster.html"
    m.save(output_path)

    print(f"Mapa de pontos gerado com sucesso! Salvo como {output_path}")

except Exception as e:
    print(f"Erro ao gerar o mapa de pontos: {e}")


Mapa de pontos gerado com sucesso! Salvo como C:/Users/gusta/Desktop/TCC Gustavo/notebooks/mapa_de_pontos_cluster.html


municipios e trexos com maior numero de acidentes 


In [ ]:

accidents_by_municipio = dataSet['municipio'].value_counts().head(10).reset_index()
accidents_by_municipio.columns = ['Municipio', 'Num_Acidentes']


dataSet['km'] = dataSet['km'].replace(',', '.', regex=True).astype(float)  
accidents_by_road_section = dataSet.groupby(['br', 'km']).size().reset_index(name='Num_Acidentes')
accidents_by_road_section = accidents_by_road_section.sort_values(by='Num_Acidentes', ascending=False).head(10)
accidents_by_road_section.columns = ['Rodovia', 'Km', 'Num_Acidentes']

print("Acidentes por Município")
print(accidents_by_municipio)

print("\nAcidentes por Trecho da Rodovia")
print(accidents_by_road_section)


Acidentes por Município
          Municipio  Num_Acidentes
0             SERRA           6789
1          LINHARES           3552
2         CARIACICA           3316
3             VIANA           2160
4        SAO MATEUS           1840
5  DOMINGOS MARTINS           1554
6           IBATIBA           1259
7          COLATINA           1003
8         SOORETAMA            989
9          ANCHIETA            977

Acidentes por Trecho da Rodovia
      Rodovia     Km  Num_Acidentes
1042    101.0  270.0            459
2492    262.0  150.1            360
1022    101.0  268.0            299
1240    101.0  297.0            251
1032    101.0  269.0            246
514     101.0  149.0            223
499     101.0  147.0            219
480     101.0  145.0            201
489     101.0  146.0            186
2151    262.0   31.5            167


Padrões de associação com FP-Growth

In [ ]:

from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

selected_columns = ['causa_principal', 'tipo_acidente', 'condicao_metereologica', 'fase_dia', 'tipo_pista', 'uso_solo']
data_fp = pd.get_dummies(dataSet[selected_columns])

min_support = 0.05

frequent_patterns = fpgrowth(data_fp, min_support=min_support, use_colnames=True)

rules = association_rules(frequent_patterns, metric="lift", min_threshold=1.2)

print("Padrões Frequentes de Acidentes")
print(frequent_patterns.head(10))

print("\nRegras de Associação entre Causas e Tipos de Acidentes")
print(rules.head(10))


Padrões Frequentes de Acidentes
    support                                      itemsets
0  0.758465                         (causa_principal_Sim)
1  0.579685            (condicao_metereologica_Céu Claro)
2  0.568776                          (tipo_pista_Simples)
3  0.557751                                (uso_solo_Não)
4  0.331883                        (fase_dia_Plena Noite)
5  0.068413     (tipo_acidente_Saída de leito carroçável)
6  0.158383              (tipo_acidente_Colisão traseira)
7  0.564134                          (fase_dia_Pleno dia)
8  0.056199            (tipo_acidente_Colisão com objeto)
9  0.121653  (tipo_acidente_Queda de ocupante de veículo)

Regras de Associação entre Causas e Tipos de Acidentes
                                         antecedents  \
0                               (tipo_pista_Simples)   
1                                     (uso_solo_Não)   
2          (tipo_pista_Simples, causa_principal_Sim)   
3                (causa_principal_Sim, uso_solo_Nã

#### Associação entre Tipo de Veículo e Gravidade do Acidente
Insight Potencial: Alguns tipos de veículos, como motocicletas e caminhões, podem estar associados a acidentes mais graves devido às suas características. Identificar essa relação pode ser útil para campanhas de conscientização e treinamento de motoristas.

Regras de Associação: Se tipo_veiculo é "Motocicleta", então a probabilidade de classificacao_acidente ser "Com Vítimas Fatais" é alta.

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

selected_columns = ['tipo_veiculo', 'classificacao_acidente']

data_fp = pd.get_dummies(data[selected_columns])

frequent_patterns = fpgrowth(data_fp, min_support=0.02, use_colnames=True)

rules = association_rules(frequent_patterns, metric="confidence", min_threshold=0.5)
rules = rules[(rules['lift'] > 1) & (rules['consequents'].astype(str).str.contains("classificacao_acidente"))]

rules_visualization = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'leverage', 'conviction']]

rules_visualization = rules_visualization.sort_values(by=['confidence', 'lift'], ascending=[False, False]).reset_index(drop=True)

print("Top 10 Regras de Associação entre Tipo de Veículo e Gravidade do Acidente:")
print(rules_visualization.head(10))


Top 10 Regras de Associação entre Tipo de Veículo e Gravidade do Acidente:
                  antecedents                                   consequents  \
0     (tipo_veiculo_Motoneta)  (classificacao_acidente_Com Vítimas Feridas)   
1  (tipo_veiculo_Motocicleta)  (classificacao_acidente_Com Vítimas Feridas)   
2       (tipo_veiculo_Ônibus)  (classificacao_acidente_Com Vítimas Feridas)   

    support  confidence      lift  leverage  conviction  
0  0.028897    0.955854  1.156386  0.003908    3.928164  
1  0.217367    0.920732  1.113896  0.022226    2.187683  
2  0.022253    0.849391  1.027587  0.000597    1.151406  


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

data_encoded = data.apply(LabelEncoder().fit_transform)

X = data_encoded.drop('classificacao_acidente', axis=1)
y = data_encoded['classificacao_acidente']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

feature_importances = pd.DataFrame({
    'variavel': X.columns,
    'importancia': model.feature_importances_
}).sort_values(by='importancia', ascending=False)

print("Importância das Variáveis:")
print(feature_importances)


c:\Users\gusta\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\gusta\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\gusta\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\gusta\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not 

Importância das Variáveis:
                  variavel  importancia
24           estado_fisico     0.071647
30                  mortos     0.070661
4                  horario     0.063178
32               longitude     0.055165
31                latitude     0.051192
0                       id     0.047971
7                       km     0.047389
19              id_veiculo     0.046159
2             data_inversa     0.045183
21                   marca     0.044864
1                    pesid     0.040481
12           tipo_acidente     0.035721
22  ano_fabricacao_veiculo     0.033076
8                municipio     0.028868
10          causa_acidente     0.027853
25                   idade     0.027789
3               dia_semana     0.025726
11     ordem_tipo_acidente     0.024094
17             tracado_via     0.023624
28           feridos_leves     0.022584
20            tipo_veiculo     0.021653
15  condicao_metereologica     0.015937
23          tipo_envolvido     0.015209
13           

In [ ]:
import pandas as pd
import unidecode

def remove_acentos(texto):
    return unidecode.unidecode(texto)

file_path_corrigido = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23.csv'

df_pandas = pd.read_csv(file_path_corrigido, delimiter=';', encoding='ISO-8859-1')

print("Colunas disponíveis no dataset:", df_pandas.columns.tolist())

df_pandas = df_pandas.applymap(lambda x: remove_acentos(str(x)))

output_file = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23_final.csv'
df_pandas.to_csv(output_file, index=False, encoding='ISO-8859-1')

print(f"Arquivo corrigido e transformado salvo em: {output_file}")


Colunas disponíveis no dataset: ['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_principal', 'causa_acidente', 'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude', 'regional', 'delegacia', 'uop']


C:\Users\gusta\AppData\Local\Temp\ipykernel_5332\2476256370.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pandas = df_pandas.applymap(lambda x: remove_acentos(str(x)))


Arquivo corrigido e transformado salvo em: C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23_final.csv


In [ ]:
import pandas as pd

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23.csv'

df_pandas = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1')

df_pandas.columns = df_pandas.columns.str.strip()

print("Colunas disponíveis no dataset:", df_pandas.columns.tolist())

if 'condicao_metereologica' in df_pandas.columns:
    print("Valores únicos na coluna 'condicao_metereologica':")
    print(df_pandas['condicao_metereologica'].unique())
else:
    print("A coluna 'condicao_metereologica' não foi encontrada!")

output_file = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23_cleaned.csv'
df_pandas.to_csv(output_file, index=False, encoding='ISO-8859-1')

print(f"Arquivo limpo e corrigido salvo em: {output_file}")


Colunas disponíveis no dataset: ['id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,causa_acidente,ordem_tipo_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,id_veiculo,tipo_veiculo,marca,ano_fabricacao_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,"uop,,,"']
A coluna 'condicao_metereologica' não foi encontrada!
Arquivo limpo e corrigido salvo em: C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23_cleaned.csv


In [ ]:
import pandas as pd

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23.csv'

try:
    df_comma = pd.read_csv(file_path, delimiter=',', encoding='ISO-8859-1')
    print("Delimitador vírgula (',') funciona!")
    print(df_comma.head())
except Exception as e:
    print("Erro ao tentar ler com vírgula:", e)

try:
    df_semicolon = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1')
    print("Delimitador ponto e vírgula (';') funciona!")
    print(df_semicolon.head())
except Exception as e:
    print("Erro ao tentar ler com ponto e vírgula:", e)


Delimitador vírgula (',') funciona!
        id      pesid data_inversa   dia_semana     horario  uf      br  \
0  3317220  7399070.0     1/1/2021  sexta-feira  4:10:00 AM  ES  1010.0   
1  3317220  7399030.0     1/1/2021  sexta-feira  4:10:00 AM  ES  1010.0   
2  3317220  7399070.0     1/1/2021  sexta-feira  4:10:00 AM  ES  1010.0   
3  3317220  7399030.0     1/1/2021  sexta-feira  4:10:00 AM  ES  1010.0   
4  3317220  7399070.0     1/1/2021  sexta-feira  4:10:00 AM  ES  1010.0   

      km   municipio causa_principal  ...       sexo  ilesos feridos_leves  \
0  527.0  SAO MATEUS             Sim  ...  Masculino       1             0   
1  527.0  SAO MATEUS             Sim  ...  Masculino       1             0   
2  527.0  SAO MATEUS             Sim  ...  Masculino       1             0   
3  527.0  SAO MATEUS             Sim  ...  Masculino       1             0   
4  527.0  SAO MATEUS             Sim  ...  Masculino       1             0   

  feridos_graves mortos    latitude   longit

In [ ]:
print("Colunas disponíveis no dataset:")
print(dataSet.columns.tolist())


Colunas disponíveis no dataset:
['id;pesid;data_inversa;dia_semana;horario;uf;br;km;municipio;causa_principal;causa_acidente;ordem_tipo_acidente;tipo_acidente;classificacao_acidente;fase_dia;sentido_via;condicao_metereologica;tipo_pista;tracado_via;uso_solo;id_veiculo;tipo_veiculo;marca;ano_fabricacao_veiculo;tipo_envolvido;estado_fisico;idade;sexo;ilesos;feridos_leves;feridos_graves;mortos;latitude;longitude;regional;delegacia;uop']


mlxtend

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

def processar_dataset(file_path, min_support, min_confidence):
    
    df_pandas = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1')

    
    df_pandas.columns = df_pandas.columns.str.strip()

    
    columns_to_use = ['causa_acidente','tipo_acidente','classificacao_acidente','fase_dia',
                      'condicao_metereologica','tipo_veiculo','sexo','tracado_via',
                      'idade','sentido_via','horario','dia_semana','estado_fisico','municipio']

    
    df_pandas = df_pandas[columns_to_use]

    
    df_pandas['items'] = df_pandas[columns_to_use].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

    df_onehot = df_pandas['items'].str.get_dummies(sep=',')

    frequent_itemsets = fpgrowth(df_onehot, min_support=min_support, use_colnames=True)

    association_rules_result = association_rules(frequent_itemsets, metric="lift", min_threshold=min_confidence)

    association_rules_filtered = association_rules_result[association_rules_result['confidence'] >= min_confidence]

    num_regras = len(association_rules_filtered)
    print(f"✅ Número de regras geradas com confiança acima de {min_confidence}%: {num_regras}")

    print("🔗 Regras de Associação acima de 80% de confiança:")
    print(association_rules_filtered)

    output_frequent_items = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/frequent_items_mlxtend.csv"
    output_association_rules = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/association_rules_mlxtend.csv"

    frequent_itemsets.to_csv(output_frequent_items, index=False, encoding='ISO-8859-1')
    association_rules_filtered.to_csv(output_association_rules, index=False, encoding='ISO-8859-1')

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23.csv'

min_support = 0.02  # Suporte mínimo
min_confidence = 0.9  # Confiança mínima

processar_dataset(file_path, min_support, min_confidence)


c:\Users\gusta\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


✅ Número de regras geradas com confiança acima de 0.9%: 852
🔗 Regras de Associação acima de 80% de confiança:
                                           antecedents            consequents  \
11102            (Motocicleta, sexta-feira, Masculino)  (Com Vítimas Feridas)   
11129                 (Motocicleta, Reta, sexta-feira)  (Com Vítimas Feridas)   
11158            (Motocicleta, sexta-feira, Pleno dia)  (Com Vítimas Feridas)   
11214                               (nan, sexta-feira)        (Não Informado)   
11226                      (Lesões Leves, sexta-feira)  (Com Vítimas Feridas)   
...                                                ...                    ...   
59166           (Motocicleta, Masculino, quinta-feira)  (Com Vítimas Feridas)   
59178                (Motocicleta, Reta, quinta-feira)  (Com Vítimas Feridas)   
59465                 (Masculino, SERRA, quinta-feira)  (Com Vítimas Feridas)   
59658  (quinta-feira, Crescente, Masculino, Pleno dia)  (Com Vítimas Feridas)   

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules


def processar_dataset(file_path, min_support, min_confidence):
    
    df_pandas = pd.read_csv(file_path, delimiter=',', encoding='ISO-8859-1')

    df_pandas.columns = df_pandas.columns.str.strip()

    columns_to_use = ['causa_acidente','tipo_acidente','classificacao_acidente','fase_dia','condicao_metereologica','tipo_veiculo','sexo','tracado_via','idade','horario','dia_semana','estado_fisico','municipio']

    df_pandas = df_pandas[columns_to_use]

    df_pandas = df_pandas.replace("Não Informado", pd.NA).dropna()

    df_pandas['items'] = df_pandas[columns_to_use].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

    df_onehot = df_pandas['items'].str.get_dummies(sep=',')

    frequent_itemsets = fpgrowth(df_onehot, min_support=min_support, use_colnames=True)

    association_rules_result = association_rules(frequent_itemsets, metric="lift", min_threshold=min_confidence)

    association_rules_filtered = association_rules_result[association_rules_result['confidence'] >= min_confidence]

    association_rules_filtered['antecedents'] = association_rules_filtered['antecedents'].apply(lambda x: ', '.join(list(x)))
    association_rules_filtered['consequents'] = association_rules_filtered['consequents'].apply(lambda x: ', '.join(list(x)))

    num_regras = len(association_rules_filtered)
    print(f"✅ Número de regras geradas com confiança acima de {min_confidence}%: {num_regras}")

    
    print("🔗 Regras de Associação ")
    print(association_rules_filtered)


    output_frequent_items = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/frequent_items_mlxtend.csv"
    output_association_rules = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/association_rules_mlxtend.csv"

    frequent_itemsets.to_csv(output_frequent_items, index=False, encoding='ISO-8859-1')
    association_rules_filtered.to_csv(output_association_rules, index=False, encoding='ISO-8859-1')

file_path = 'C:/Users/gusta/Desktop/TCC Gustavo/notebooks/acidentesES21,22,23.csv'

min_support = 0.02  # Suporte mínimo
min_confidence = 0.8  # Confiança mínima


processar_dataset(file_path, min_support, min_confidence)


c:\Users\gusta\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


✅ Número de regras geradas com confiança acima de 0.8%: 2621
🔗 Regras de Associação 
                antecedents          consequents  antecedent support  \
1                 Masculino  Com Vítimas Feridas            0.760397   
4                      Reta  Com Vítimas Feridas            0.665685   
8           Reta, Masculino  Com Vítimas Feridas            0.508036   
16                Céu Claro  Com Vítimas Feridas            0.599926   
19     Céu Claro, Masculino  Com Vítimas Feridas            0.461661   
...                     ...                  ...                 ...   
49107            JOAO NEIVA  Com Vítimas Feridas            0.022451   
49108              ANCHIETA  Com Vítimas Feridas            0.033738   
49112       Curva, ANCHIETA  Com Vítimas Feridas            0.024782   
49124              COLATINA  Com Vítimas Feridas            0.027849   
49126              COLATINA            Masculino            0.027849   

       consequent support   support  confidence   

C:\Users\gusta\AppData\Local\Temp\ipykernel_9508\1557000427.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  association_rules_filtered['antecedents'] = association_rules_filtered['antecedents'].apply(lambda x: ', '.join(list(x)))
C:\Users\gusta\AppData\Local\Temp\ipykernel_9508\1557000427.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  association_rules_filtered['consequents'] = association_rules_filtered['consequents'].apply(lambda x: ', '.join(list(x)))


In [ ]:
import pandas as pd

file_path_rules = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/association_rules_mlxtend.csv"

rules_df = pd.read_csv(file_path_rules, encoding="ISO-8859-1")

rules_df_sorted = rules_df.sort_values(by="confidence", ascending=False)

print(rules_df_sorted.head(10))

output_path_sorted = "C:/Users/gusta/Desktop/TCC Gustavo/notebooks/association_rules_sorted_by_confidence.csv"
rules_df_sorted.to_csv(output_path_sorted, index=False, encoding="ISO-8859-1")

print(f" Regras ordenadas por confiança salvas em: {output_path_sorted}")


        ID                               antecedents          consequents  \
2187  2188  Motocicleta, Lesões Leves, segunda-feira  Com Vítimas Feridas   
2504  2505     Lesões Leves, quinta-feira, Pleno dia  Com Vítimas Feridas   
2474  2475    Lesões Leves, Sol, Com Vítimas Feridas            Pleno dia   
1334  1335       Motocicleta, Lesões Leves, Feminino  Com Vítimas Feridas   
319    320              Reta, Masculino, Sem Vítimas                Ileso   
2475  2476              Lesões Leves, Sol, Pleno dia  Com Vítimas Feridas   
317    318                         Reta, Sem Vítimas                Ileso   
2425  2426                            Sol, Masculino            Pleno dia   
1282  1283          Interseção de vias, Lesões Leves  Com Vítimas Feridas   
2473  2474                         Lesões Leves, Sol            Pleno dia   

      antecedent support  consequent support   support  confidence      lift  \
2187            0.023923            0.849834  0.023923         1.0  1.17

In [ ]:
import pandas as pd

def adjust_time_interval(time_str):
    try:
        time = pd.to_datetime(time_str, format='%H:%M:%S', errors='coerce')
    except Exception as e:
        print(f"Erro ao converter o horário: {time_str}, erro: {e}")
        return None

    if pd.isna(time):
        return None

    minute = time.minute
    if minute < 15:
        time = time.replace(minute=0, second=0)
    elif 15 <= minute < 45:
        time = time.replace(minute=30, second=0)
    else:
        time = time.replace(minute=0, second=0) + pd.Timedelta(hours=1)

    return time.strftime('%H:%M:%S')

file_path = r'C:\Users\gusta\Desktop\TCC Gustavo\datasets gerados BI\Nº de ocorrência por hora.csv'  # Caminho normalizado

try:
    data = pd.read_csv(file_path, sep=',', header=0) 
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    exit()

if data.empty:
    print("O arquivo CSV está vazio ou não foi lido corretamente.")
    exit()

required_columns = ['Horario', 'Contagem de Ocorrencia', 'Ano']
if not all(col in data.columns for col in required_columns):
    print(f"Colunas necessárias estão ausentes. Esperado: {required_columns}, Encontrado: {data.columns.tolist()}")
    exit()

data = data.dropna(subset=['Horario'])

data['Horario_Ajustado'] = data['Horario'].apply(adjust_time_interval)

data = data.dropna(subset=['Horario_Ajustado'])

data_grouped = data.groupby(['Horario_Ajustado', 'Ano'], as_index=False)['Contagem de Ocorrencia'].sum()

data_pivot = data_grouped.pivot(index='Horario_Ajustado', columns='Ano', values='Contagem de Ocorrencia').reset_index()
data_pivot.columns.name = None  

if data_pivot.empty:
    print("Nenhum dado foi gerado após o processamento.")
else:
    print(data_pivot.head())

output_path = file_path.replace('.csv', '_ajustado.csv')
data_pivot.to_csv(output_path, index=False)
print(f"Arquivo ajustado salvo em: {output_path}")


  Horario_Ajustado  2021  2022  2023
0         00:00:00   130   104    77
1         00:30:00    75    67    67
2         01:00:00    82    66    46
3         01:30:00    33   124    45
4         02:00:00    45    59    76
Arquivo ajustado salvo em: C:\Users\gusta\Desktop\TCC Gustavo\datasets gerados BI\Nº de ocorrência por hora_ajustado.csv
